In [21]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from aac_crud import AnimalShelterCRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

# collection variables
username = "aacuser"
password = "Disney_Rules"
HOST = 'nv-desktop-services.apporto.com'
PORT = 31948
DB = 'AAC'
COL = 'animals'
shelter = AnimalShelterCRUD(username, password, HOST, PORT, DB, COL)

# Connect to database via CRUD Module
df = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column.
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
app.layout = html.Div([
     html.Div(className='row', style={'display': 'flex', 'margin': '5px 0', 'alignItems': 'center'}, children=[
         # Wrap the logo in an anchor tag for the clickable link
        html.A(
            href="https://www.snhu.edu",  # Link to the specified URL
            target="_blank",  # Open the link in a new tab
            children=html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                style={'height': '20%', 'width': '20%'}
            )
        ),
        html.Center(html.B(html.H1('Grazioso Salvare Dashboard - CS-340 Courtney Warner', style={'fontSize': '24px', 'marginLeft': '10px'})))
            
    ]),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': ' Water Rescue ', 'value': 'Water'},
                {'label': ' Mountain and Wilderness Rescue ', 'value': 'Mountain'},
                {'label': ' Disaster and Individual Rescue ', 'value': 'Disaster'},
                {'label': ' Reset', 'value': 'Reset'},
            ],
            value='Reset'
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable='single',
        row_selectable='single',  # Single row selection for mapping callback
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],  # Set for the default selection
        page_action='native',
        page_current=0,
        page_size=10,
        
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type is None:
        return []
    
    df = None
    if filter_type == 'Water':
        df = pd.DataFrame.from_records(shelter.read({
            "$and": [
                {"$or": [
                    {'breed': {"$regex": 'Labrador Retriever Mix'}},
                    {'breed': {"$regex": 'Chesapeake'}},
                    {'breed': {"$regex": 'Newfoundland'}}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {"$gt": 26, "$lt": 156}}
            ]
        }))
    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(shelter.read({
            "$and": [
                {"$or": [
                    {'breed': {"$regex": 'German Shepherd'}},
                    {'breed': {"$regex": 'Alaskan Malamute'}},
                    {'breed': {"$regex": 'Old English Sheepdog'}},
                    {'breed': {"$regex": 'Siberian Husky'}},
                    {'breed': {"$regex": 'Rottweiler'}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {"$gt": 26, "$lt": 156}}
            ]
        }))
    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(shelter.read({
            "$and": [
                {"$or": [
                    {'breed': {"$regex": 'Doberman Pinscher'}},
                    {'breed': {"$regex": 'German Shepherd'}},
                    {'breed': {"$regex": 'Golden Retriever'}},
                    {'breed': {"$regex": 'Bloodhound'}},
                    {'breed': {"$regex": 'Rottweiler'}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {"$gt": 20, "$lt": 300}}
            ]
        }))
    else:
        df = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}))
    
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    
    df = pd.DataFrame(viewData)
    return [
        dcc.Graph(
            figure=px.pie(df, names='breed', title='Preferred Animals')
        )
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []
    
    dff = pd.DataFrame(viewData)
    row = index[0] if index else 0
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])
            ])
        ])
    ]

# Run the app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:28242/
